In [8]:
import numpy as np
import matplotlib.pyplot as plt


import jax
import jax.numpy as jnp
import numpyro
import numpyro.distributions as dist
from jax import random

from numpyro.handlers import seed, trace
from numpyro.infer import SVI, Trace_ELBO

In [5]:
i_range = np.arange(20)
j_range = np.arange(10)

data = i_range[:, None] + j_range[None, :]
data

array([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9],
       [ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10],
       [ 2,  3,  4,  5,  6,  7,  8,  9, 10, 11],
       [ 3,  4,  5,  6,  7,  8,  9, 10, 11, 12],
       [ 4,  5,  6,  7,  8,  9, 10, 11, 12, 13],
       [ 5,  6,  7,  8,  9, 10, 11, 12, 13, 14],
       [ 6,  7,  8,  9, 10, 11, 12, 13, 14, 15],
       [ 7,  8,  9, 10, 11, 12, 13, 14, 15, 16],
       [ 8,  9, 10, 11, 12, 13, 14, 15, 16, 17],
       [ 9, 10, 11, 12, 13, 14, 15, 16, 17, 18],
       [10, 11, 12, 13, 14, 15, 16, 17, 18, 19],
       [11, 12, 13, 14, 15, 16, 17, 18, 19, 20],
       [12, 13, 14, 15, 16, 17, 18, 19, 20, 21],
       [13, 14, 15, 16, 17, 18, 19, 20, 21, 22],
       [14, 15, 16, 17, 18, 19, 20, 21, 22, 23],
       [15, 16, 17, 18, 19, 20, 21, 22, 23, 24],
       [16, 17, 18, 19, 20, 21, 22, 23, 24, 25],
       [17, 18, 19, 20, 21, 22, 23, 24, 25, 26],
       [18, 19, 20, 21, 22, 23, 24, 25, 26, 27],
       [19, 20, 21, 22, 23, 24, 25, 26, 27, 28]])

In [39]:
def model(data):

    ip = numpyro.plate('i', data.shape[0], dim=-2)
    jp = numpyro.plate('j', data.shape[1], dim=-1)
    with ip:
        a = numpyro.sample('a', dist.Uniform(0,30))
    a = a.reshape((20,1))
    # print(a.shape)
    with jp:
        b = numpyro.sample('b', dist.Uniform(0,30))
    b = b.reshape((1,10))
    # print(b.shape)
    # print((a+b).shape)
    with ip, jp:
        numpyro.sample('y', dist.Normal(a + b, 1.), obs=data)

In [40]:
import pprint as pp

rng_key = random.PRNGKey(0)
exec_trace = trace(seed(model, rng_key)).get_trace(data)
pp.pprint(exec_trace)

OrderedDict([('i',
              {'args': (20, None),
               'cond_indep_stack': [],
               'fn': <function _subsample_fn at 0x16be58c20>,
               'kwargs': {'rng_key': None},
               'name': 'i',
               'scale': 1.0,
               'type': 'plate',
               'value': Array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19], dtype=int32)}),
             ('j',
              {'args': (10, None),
               'cond_indep_stack': [],
               'fn': <function _subsample_fn at 0x16be58c20>,
               'kwargs': {'rng_key': None},
               'name': 'j',
               'scale': 1.0,
               'type': 'plate',
               'value': Array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int32)}),
             ('a',
              {'args': (),
               'cond_indep_stack': [CondIndepStackFrame(name='i', dim=-2, size=20)],
               'fn': <numpyro.distributions.distribution.ExpandedDistribution ob